# `jdbc_fast` Test 1: de-composed, step-by-step

## Setup

In [1]:
import numpy as np
import pandas as pd

Load magic:

In [2]:
%load_ext jdbc_magic

sqlmagic loaded


## Tests

Connect:

Note: if the database doesn't exist, the H2 server needs to be started with the [`-ifNotExists` flag](http://h2database.com/html/tutorial.html?highlight=ifNotExists&search=ifnotexist#firstFound).

In [3]:
%%connect

jclassname=org.h2.Driver
url=jdbc:h2:tcp://localhost:9092/./test/db
driver_args=sa;
jars=C:/Users/corne/Databases/h2-1.4.199.jar;C:\Users\corne\Eclipse\MSI\java-rsagg2\deploy\java-rsagg-1.jar

connected to dbMeta0: conn0: url=jdbc:h2:tcp://localhost:9092/./test/db user=SA


In [4]:
import jpype, jpype.imports
from jpype.types import *

In [5]:
datadict = {
    'CHAR': lambda o: np.array(o.getStr()[:]),
    'NCHAR': lambda o: np.array(o.getStr()[:]),
    'VARCHAR': lambda o: np.array(o.getStr()[:]),
    'NVARCHAR': lambda o: np.array(o.getStr()[:]),
    'LONGVARCHAR': lambda o: np.array(o.getStr()[:]),
    'LONGNVARCHAR': lambda o: np.array(o.getStr()[:]),
    'OTHER': lambda o: o.getObj()[:],
    'TINYINT': lambda o: o.getByte()[:],
    'SMALLINT': lambda o: o.getShort()[:],
    'INTEGER': lambda o: o.getInt()[:],
    'BIGINT': lambda o: o.getLong()[:],
    'REAL': lambda o: o.getFloat()[:],
    'FLOAT': lambda o: o.getDouble()[:],
    'DOUBLE': lambda o: o.getDouble()[:],
    'BOOLEAN': lambda o: o.getBoolean()[:],
    'TIME': lambda o: np.array(list(map(str, o.getTime()[:]))),
    'DATE': lambda o: o.getLong()[:].astype('datetime64[D]'),
    'TIMESTAMP': lambda o: o.getLong()[:].astype('datetime64[s]')
}

In [6]:
%%timeit

cursor.execute('SELECT * FROM test')
rsaggclazz = jpype.JClass("rsagg.RsAgg")
rsagg = rsaggclazz(cursor._rs)


try:
    rsagg.agg()
except java.lang.Exception as ex:
    print("Caught the runtime exception : ", str(ex))
    print(ex.stacktrace())

df1 = pd.DataFrame(columns=[col.name for col in rsagg.cols])
for col in rsagg.cols:
    df1[col.name] = datadict[col.typeName](col)

2.3 s ± 164 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
cursor.execute('SELECT * FROM test')
rsaggclazz = jpype.JClass("rsagg.RsAgg")
rsagg = rsaggclazz(cursor._rs)

import java.lang.Exception
try:
    rsagg.agg()
except java.lang.Exception as ex:
    print("Caught the runtime exception : ", str(ex))
    print(ex.stacktrace())

df1 = pd.DataFrame(columns=[col.name for col in rsagg.cols])
for col in rsagg.cols:
    df1[col.name] = datadict[col.typeName](col)

In [7]:
df1.dtypes

VRCH                object
TNYNT                 int8
SMLLNT               int16
NTGR                 int32
BGNT                 int64
FLT                float32
DBL                float64
BOOL                  bool
TIME                object
DATE        datetime64[ns]
DATETIME    datetime64[ns]
dtype: object

In [8]:
df1

,VRCH,TNYNT,SMLLNT,NTGR,BGNT,FLT,DBL,BOOL,TIME,DATE,DATETIME
0,0.8570693434646546,1,1,84713,494677,0.785947,0.645894,True,15:13:34,2019-12-13,2019-12-13 15:13:34
1,0.657229676912304,0,1,35,379337,0.090745,0.493027,False,15:13:34,2019-12-13,2019-12-13 15:13:34
2,0.327632998612825,0,1,52558,6132,0.399449,0.608792,True,15:13:34,2019-12-13,2019-12-13 15:13:34
3,0.02251986847331744,1,1,12459,365381,0.562153,0.892732,True,15:13:34,2019-12-13,2019-12-13 15:13:34
4,0.0964279877814328,0,1,34430,597181,0.185261,0.160332,False,15:13:34,2019-12-13,2019-12-13 15:13:34
...,...,...,...,...,...,...,...,...,...,...,...
99995,0.804104440140236,1,1,73195,797108,0.057940,0.831827,False,15:13:36,2019-12-13,2019-12-13 15:13:35
99996,0.9582865751556441,0,0,89081,156555,0.619930,0.650169,False,15:13:36,2019-12-13,2019-12-13 15:13:35
99997,0.5320675705622763,1,1,97194,646108,0.119473,0.797357,True,15:13:36,2019-12-13,2019-12-13 15:13:35
99998,0.34933872996621584,0,1,33062,558616,0.689193,0.398846,True,15:13:36,2019-12-13,2019-12-13 15:13:35


In [10]:
%%timeit -r3

cursor.execute('SELECT * FROM test')
col_names = []
for col in range(1, cursor._meta.getColumnCount() + 1):
    col_names.append(cursor._meta.getColumnName(col))
data = cursor.fetchall()
df2 = pd.DataFrame(data=data, columns=col_names if col_names else None)

26.5 s ± 815 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [11]:
cursor.execute('SELECT * FROM test')
col_names = []
for col in range(1, cursor._meta.getColumnCount() + 1):
    col_names.append(cursor._meta.getColumnName(col))
data = cursor.fetchall()
df2 = pd.DataFrame(data=data, columns=col_names if col_names else None)

In [12]:
df2.dtypes

VRCH         object
TNYNT         int64
SMLLNT        int64
NTGR          int64
BGNT          int64
FLT         float64
DBL         float64
BOOL          int64
TIME         object
DATE         object
DATETIME     object
dtype: object

In [13]:
df2

,VRCH,TNYNT,SMLLNT,NTGR,BGNT,FLT,DBL,BOOL,TIME,DATE,DATETIME
0,0.8570693434646546,1,1,84713,494677,0.785947,0.645894,1,15:13:34,2019-12-13,2019-12-13 15:13:34.414405
1,0.657229676912304,0,1,35,379337,0.090745,0.493027,0,15:13:34,2019-12-13,2019-12-13 15:13:34.414405
2,0.327632998612825,0,1,52558,6132,0.399449,0.608792,1,15:13:34,2019-12-13,2019-12-13 15:13:34.414405
3,0.02251986847331744,1,1,12459,365381,0.562153,0.892732,1,15:13:34,2019-12-13,2019-12-13 15:13:34.414405
4,0.0964279877814328,0,1,34430,597181,0.185261,0.160332,0,15:13:34,2019-12-13,2019-12-13 15:13:34.414405
...,...,...,...,...,...,...,...,...,...,...,...
99995,0.804104440140236,1,1,73195,797108,0.057940,0.831827,0,15:13:36,2019-12-13,2019-12-13 15:13:35.580262
99996,0.9582865751556441,0,0,89081,156555,0.619930,0.650169,0,15:13:36,2019-12-13,2019-12-13 15:13:35.580262
99997,0.5320675705622763,1,1,97194,646108,0.119473,0.797357,1,15:13:36,2019-12-13,2019-12-13 15:13:35.580262
99998,0.34933872996621584,0,1,33062,558616,0.689193,0.398846,1,15:13:36,2019-12-13,2019-12-13 15:13:35.580262


Close:

In [9]:
%close

connection closed
